# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-16 23:06:19] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=38367, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=745054848, constrai

[2025-07-16 23:06:31] Attention backend not set. Use fa3 backend by default.
[2025-07-16 23:06:31] Init torch distributed begin.


[2025-07-16 23:06:32] Init torch distributed ends. mem usage=0.00 GB


[2025-07-16 23:06:33] Load weight begin. avail mem=78.50 GB


[2025-07-16 23:06:33] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-16 23:06:37] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=47.86 GB


[2025-07-16 23:06:37] INFO:     Started server process [3015862]
[2025-07-16 23:06:37] INFO:     Waiting for application startup.
[2025-07-16 23:06:37] INFO:     Application startup complete.
[2025-07-16 23:06:37] INFO:     Uvicorn running on http://0.0.0.0:38367 (Press CTRL+C to quit)
[2025-07-16 23:06:37] INFO:     127.0.0.1:38806 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-16 23:06:38] INFO:     127.0.0.1:38814 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-16 23:06:38] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:38.514609


[2025-07-16 23:06:39] INFO:     127.0.0.1:38818 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 23:06:39] The server is fired up and ready to roll!


Server started on http://localhost:38367


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-16 23:06:42] INFO:     127.0.0.1:52434 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-07-16 23:06:42] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:42.670876


[2025-07-16 23:06:43] INFO:     127.0.0.1:52442 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-07-16 23:06:43] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:43.111483
[2025-07-16 23:06:43] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.50, #queue-req: 0, timestamp: 2025-07-16T23:06:43.177375


[2025-07-16 23:06:43] INFO:     127.0.0.1:52454 - "POST /generate HTTP/1.1" 200 OK


[2025-07-16 23:06:43] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 74, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:43.590952
[2025-07-16 23:06:43] Decode batch. #running-req: 1, #token: 103, token usage: 0.01, cuda graph: False, gen throughput (token/s): 78.06, #queue-req: 0, timestamp: 2025-07-16T23:06:43.689797


[2025-07-16 23:06:44] INFO:     127.0.0.1:52462 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-07-16 23:06:44] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:44.161395
[2025-07-16 23:06:44] INFO:     127.0.0.1:52478 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 23:06:44] Prefill batch. #new-seq: 2, #new-token: 5, #cached-token: 62, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:44.201501
[2025-07-16 23:06:44] INFO:     127.0.0.1:52480 - "POST /generate HTTP/1.1" 200 OK


[2025-07-16 23:06:44] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:44.349062


[2025-07-16 23:06:44] Decode batch. #running-req: 1, #token: 53, token usage: 0.00, cuda graph: False, gen throughput (token/s): 52.53, #queue-req: 0, timestamp: 2025-07-16T23:06:44.470300


[2025-07-16 23:06:45] INFO:     127.0.0.1:52484 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-07-16 23:06:45] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:45.217589
[2025-07-16 23:06:45] Prefill batch. #new-seq: 1, #new-token: 35, #cached-token: 14, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-16T23:06:45.220130
[2025-07-16 23:06:45] Decode batch. #running-req: 2, #token: 72, token usage: 0.00, cuda graph: False, gen throughput (token/s): 52.53, #queue-req: 0, timestamp: 2025-07-16T23:06:45.384059


[2025-07-16 23:06:45] Decode batch. #running-req: 2, #token: 152, token usage: 0.01, cuda graph: False, gen throughput (token/s): 132.68, #queue-req: 0, timestamp: 2025-07-16T23:06:45.987034


[2025-07-16 23:06:46] Decode batch. #running-req: 2, #token: 232, token usage: 0.01, cuda graph: False, gen throughput (token/s): 132.42, #queue-req: 0, timestamp: 2025-07-16T23:06:46.591192


[2025-07-16 23:06:47] Decode batch. #running-req: 2, #token: 312, token usage: 0.02, cuda graph: False, gen throughput (token/s): 139.47, #queue-req: 0, timestamp: 2025-07-16T23:06:47.164789
[2025-07-16 23:06:47] INFO:     127.0.0.1:52504 - "POST /generate HTTP/1.1" 200 OK


[2025-07-16 23:06:47] Decode batch. #running-req: 1, #token: 218, token usage: 0.01, cuda graph: False, gen throughput (token/s): 83.23, #queue-req: 0, timestamp: 2025-07-16T23:06:47.765571
[2025-07-16 23:06:47] INFO:     127.0.0.1:52492 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 23:06:47] Prefill batch. #new-seq: 1, #new-token: 337, #cached-token: 39, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:47.825116


[2025-07-16 23:06:48] Decode batch. #running-req: 1, #token: 413, token usage: 0.02, cuda graph: False, gen throughput (token/s): 62.04, #queue-req: 0, timestamp: 2025-07-16T23:06:48.410382


[2025-07-16 23:06:48] Decode batch. #running-req: 1, #token: 453, token usage: 0.02, cuda graph: False, gen throughput (token/s): 69.15, #queue-req: 0, timestamp: 2025-07-16T23:06:48.988809


[2025-07-16 23:06:49] Decode batch. #running-req: 1, #token: 493, token usage: 0.02, cuda graph: False, gen throughput (token/s): 66.65, #queue-req: 0, timestamp: 2025-07-16T23:06:49.588932


[2025-07-16 23:06:50] Decode batch. #running-req: 1, #token: 533, token usage: 0.03, cuda graph: False, gen throughput (token/s): 66.38, #queue-req: 0, timestamp: 2025-07-16T23:06:50.191524


[2025-07-16 23:06:50] INFO:     127.0.0.1:52518 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-07-16 23:06:50] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:50.527152


[2025-07-16 23:06:52] INFO:     127.0.0.1:52524 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-07-16 23:06:53] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:53.010511


[2025-07-16 23:06:53] Decode batch. #running-req: 1, #token: 42, token usage: 0.00, cuda graph: False, gen throughput (token/s): 13.84, #queue-req: 0, timestamp: 2025-07-16T23:06:53.080953


[2025-07-16 23:06:53] Decode batch. #running-req: 1, #token: 82, token usage: 0.00, cuda graph: False, gen throughput (token/s): 68.62, #queue-req: 0, timestamp: 2025-07-16T23:06:53.663847


[2025-07-16 23:06:54] Decode batch. #running-req: 1, #token: 122, token usage: 0.01, cuda graph: False, gen throughput (token/s): 68.02, #queue-req: 0, timestamp: 2025-07-16T23:06:54.251927


[2025-07-16 23:06:54] INFO:     127.0.0.1:52774 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-07-16 23:06:54] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:54.710022
[2025-07-16 23:06:54] INFO:     127.0.0.1:52784 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:00,  7.04it/s]

 67%|██████▋   | 2/3 [00:00<00:00, 11.16it/s]

[2025-07-16 23:06:54] Prefill batch. #new-seq: 3, #new-token: 29, #cached-token: 51, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:54.745226
[2025-07-16 23:06:54] INFO:     127.0.0.1:52802 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 23:06:54] INFO:     127.0.0.1:52814 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 23:06:54] Decode batch. #running-req: 3, #token: 37, token usage: 0.00, cuda graph: False, gen throughput (token/s): 87.72, #queue-req: 0, timestamp: 2025-07-16T23:06:54.890333


100%|██████████| 3/3 [00:00<00:00, 17.50it/s]

[2025-07-16 23:06:54] INFO:     127.0.0.1:52792 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-07-16 23:06:54] INFO:     127.0.0.1:52830 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 23:06:54] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:54.920084
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-07-16 23:06:55] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:06:55.058849


[2025-07-16 23:06:55] INFO:     127.0.0.1:52838 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-16 23:06:55] Child process unexpectedly failed with exitcode=9. pid=3016309


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-07-16 23:07:07] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=36606, nccl_port=None, mem_fraction_static=0.7866, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=875478840

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-16 23:07:09] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-16 23:07:09] Inferred chat template from model path: qwen2-vl


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-07-16 23:07:19] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-07-16 23:07:20] Attention backend not set. Use flashinfer backend by default.
[2025-07-16 23:07:20] Init torch distributed begin.
[2025-07-16 23:07:20] Init torch distributed ends. mem usage=0.00 GB


...

[2025-07-16 23:07:26] INFO:     Started server process [3018793]
[2025-07-16 23:07:26] INFO:     Waiting for application startup.
[2025-07-16 23:07:26] INFO:     Application startup complete.
[2025-07-16 23:07:26] INFO:     Uvicorn running on http://0.0.0.0:36606 (Press CTRL+C to quit)


[2025-07-16 23:07:27] INFO:     127.0.0.1:38708 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-16 23:07:28] INFO:     127.0.0.1:38714 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-16 23:07:28] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:07:28.003526


[2025-07-16 23:07:29] INFO:     127.0.0.1:38728 - "POST /generate HTTP/1.1" 200 OK
[2025-07-16 23:07:29] The server is fired up and ready to roll!


Server started on http://localhost:36606


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-07-16 23:07:32] INFO:     127.0.0.1:35980 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-07-16 23:07:33] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-16T23:07:33.048204


[2025-07-16 23:07:33] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: False, gen throughput (token/s): 5.41, #queue-req: 0, timestamp: 2025-07-16T23:07:33.868270


[2025-07-16 23:07:34] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: False, gen throughput (token/s): 61.79, #queue-req: 0, timestamp: 2025-07-16T23:07:34.515583


[2025-07-16 23:07:35] INFO:     127.0.0.1:35982 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)

[2025-07-16 23:07:35] Child process unexpectedly failed with exitcode=9. pid=3019211
[2025-07-16 23:07:35] Child process unexpectedly failed with exitcode=9. pid=3018947
